<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/MachineLearningTheory/blob/master/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=678aad27596e7972d508b136c66e5b3fd753a4625f97987911d11ea0824062a1
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
import yfinance as yf
import numpy as np

need to add a global variable self.method. so we can make suitable gradient in backprop function

In [ ]:
class dynamic_NN:

  def __init__(self, X, Y, **kwargs):
    
    #usr should explicitly define layer0 to last layer
    self.__NN_layer_dict = kwargs
    self.__Y = Y
    #theta is dict with 'layer0': 5*3 ...
    self.__theta = self.construct_theta()
    self.__forward_prop = {'layer0':X}
    self.__m = len(Y)

  def construct_theta(self):

    theta = {}
    epsilon_init = 0.12
    for layer in range(len(self.__NN_layer_dict)-1):
      theta['layer'+str(layer)] = np.random.rand(self.__NN_layer_dict['layer'+str(layer+1)], self.__NN_layer_dict['layer'+str(layer)]+1)*epsilon_init -epsilon_init

    return theta

  def activate(self,method,X):
    #allow user to choose activation method in each layer from layer 1
    if method == 'sigmoid':
      result = 1/(1+np.exp(-1*X))

    return result

  ##add one to the layer
  def add_ones(self,layer_X):

    _,num_sample = np.shape(layer_X)
    ones = np.ones((1,num_sample))
    result_X = np.insert(layer_X,0,ones,axis=0)

    return result_X

  def forward_propagation(self,methods):

    #for i = 1 to len(NN_dict)
    for layer_num in range(1,len(self.__NN_layer_dict)):

      #compute i layer using forward prop i-1
      preparation = self.add_ones(self.__forward_prop['layer'+str(layer_num-1)])

      Z_val = np.dot(self.__theta['layer'+str(layer_num-1)], preparation)

      #allow user to choose activation method in each layer from layer 1
      Activation = self.activate(methods['layer'+str(layer_num)],Z_val)

      self.__forward_prop['layer'+str(layer_num)] = Activation
  
  def compute_cost(self, penalty_lambda):
    
    #compute last layer hypothesis
    pos = len(self.__NN_layer_dict) - 1 #find the label of last layer
    hypos = self.__forward_prop['layer'+str(pos)]

    #compute logged hypothesis
    log_hypos = np.log(hypos)

    #compute sum theta sq
    sum_theta_sq = 0
    for layer in self.__NN_layer_dict:

      temp = self.__NN_layer_dict[layer]
      temp = np.square(temp)
      sum_theta_sq = sum_theta_sq + np.sum(temp)

    #compute regularization term
    regularization = penalty_lambda/(2*self.__m)*sum_theta_sq
    cost = -1/self.__m * (np.dot(self.__Y, log_hypos.transpose()) + np.dot(1-self.__Y, (1-log_hypos).transpose())) + regularization
    
    return cost

  def sigmoid_gradient(self,Z):
    
    gradient = np.multiply(Z,1-Z)
    return gradient

  def backward_propagation(self,penalty_lambda):

    pos = len(self.__NN_layer_dict) - 1 #find the label of last layer (if num-of-layer = 3, pos = 2) 
    hypos = self.__forward_prop['layer'+str(pos)] 

    delta_last_layer = self.__Y - hypos
    delta = {'layer'+str(pos):delta_last_layer}

    #from layer last-1 to 1 find delta(i.e from 1 to 1)
    for layer in reversed(range(1,pos)):
      delta_temp = np.dot(self.__theta['layer'+str(layer)][:,1:].transpose(), delta['layer'+str(layer+1)])
      ##if method is not sigmoid, make an if statement to select right method
      delta_temp = np.multiply(delta_temp, self.sigmoid_gradient(self.__forward_prop['layer'+str(layer)]))
      delta['layer'+str(layer)] = delta_temp

    gradient = {}
    #i.e from 0 to 1
    for layer in range(pos):
      activation = self.add_ones(self.__forward_prop['layer'+str(layer)])
      gradient_temp = 1/self.__m * np.dot(delta['layer'+str(layer+1)], activation.transpose())

      theta_no_bias = self.__theta['layer'+str(layer)][:,1:]
      gradient_temp[:,1:] = gradient_temp[:,1:] + penalty_lambda/self.__m* theta_no_bias

      gradient['layer'+str(layer)] = gradient_temp

    return gradient

  def start_learning(self,**kwargs):

    self.forward_propagation(kwargs)

    return self.__forward_prop
    

  def get_NN_layer_dict(self):

    print(self.__NN_layer_dict)

  def get_theta(self):

    print(self.__theta)

In [ ]:
  def handle_data(Y,*args):

    def __set_Y(Y):
    
      Y[Y>0] = 1
      Y[Y<0] = 0
      return Y

    Y = Y.to_numpy()
    Y_reshaped = Y.reshape((1,len(Y)))
    Y_reshaped = (Y_reshaped-Y_reshaped.mean())/np.std(Y_reshaped)
    final_Y = __set_Y(Y_reshaped)
    ##get empty x array
    X = np.array([])
    for arg in args:

      arg = arg.to_numpy()
      arg_reshaped = arg.reshape((1,len(arg)))
      arg_reshaped = (arg_reshaped-arg_reshaped.mean())/np.std(arg_reshaped)
      X = np.append(X,[arg_reshaped])

    #reshape the appended x to the shape we want
    X_reshaped = X.reshape((len(args),len(Y)))

    return X_reshaped,Y_reshaped

In [ ]:
start = '2012-12-12'
end = '2012-12-30'
Y = yf.download('AAPL',start=start,end=end,interval='1d')['Adj Close']
X = yf.download('^GSPC',start=start,end=end,interval='1d')
X_price = X['Adj Close']
X_vol = X['Volume']
X_open = X['Open']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
x,y = handle_data(Y,X_open,X_price,X_vol)

In [ ]:
test_case = dynamic_NN(x,y,layer0 = 3,layer1=4,layer2=4,layer3=1)

In [ ]:
test_case.start_learning(layer1 = 'sigmoid',layer2 = 'sigmoid', layer3 = 'sigmoid')

{'layer0': array([[-0.05794966,  0.0083743 , -0.92739695, -1.53983415,  0.21459463,
          1.90582401,  0.76798162,  1.58250261,  0.18143898, -0.18022458,
         -0.88801749, -1.06729332],
        [ 0.18309196, -0.56757215, -1.05554339,  0.33937631,  1.70520262,
          0.79243955,  1.44749227,  0.3219223 ,  0.03180028, -0.53598243,
         -0.67979528, -1.98243203],
        [ 0.4153466 ,  0.05047483, -0.09156596,  0.15782592,  1.01808766,
          0.57868487,  0.39251481,  1.95993217, -2.08435376, -1.03160181,
         -0.47767425, -0.88767107]]),
 'layer1': array([[0.46801966, 0.48369828, 0.49860117, 0.47902201, 0.43599871,
         0.44480724, 0.44556458, 0.43256029, 0.50676024, 0.50094995,
         0.49896064, 0.5246548 ],
        [0.49599283, 0.50988642, 0.54144037, 0.52812342, 0.45874113,
         0.43826579, 0.45237357, 0.45148629, 0.4993061 , 0.51625082,
         0.53419004, 0.5643338 ],
        [0.47778456, 0.48286607, 0.50223853, 0.50867287, 0.46244898,
         0.43

In [ ]:
test_case.compute_cost(1)

array([[12.46893796]])

In [ ]:
test_case.backward_propagation(1)

{'layer0': array([[-0.00036348, -0.02335872, -0.05410318, -0.05954421],
        [-0.0004777 , -0.09099017, -0.07470322, -0.00749674],
        [-0.00029143, -0.07065793,  0.00039806, -0.06479446],
        [-0.00045811, -0.08414962, -0.07621036, -0.01123966]]),
 'layer1': array([[ 0.00467148, -0.05851798, -0.01418823, -0.01845513, -0.0292495 ],
        [ 0.00864842, -0.00447989,  0.00188316, -0.10606727, -0.05340276],
        [ 0.00328682, -0.03676903, -0.03395873, -0.00517977, -0.07043221],
        [ 0.01454514, -0.05335933, -0.10449977,  0.00461669, -0.04919372]]),
 'layer2': array([[-0.57701983, -0.30748867, -0.31516397, -0.29463501, -0.34943762]])}